Se intalan librerías que quizá no se tengan con anterioridad

In [12]:
#!pip install pandas --upgrade
#!pip install nltk
#!pip install scikit-learn

Se importan las bibliotecas que se usarán

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import matplotlib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


Se converten los 3 datasets en dataframes, primero se desanida todo el código porque vienen en una lista en formato .json

In [ ]:

juegos = [] 

with open('output_steam_games.json') as file:
    for line in file: 
        data = json.loads(line)
        juegos.append(data)


df_juegos = pd.DataFrame(juegos)
df_juegos



In [ ]:

resenias = []
with open('australian_user_reviews.json', 'r', encoding = 'utf-8') as file:
    for line in file.readlines():
        resenias.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(resenias)

dfresenias = df_reviews 


In [ ]:
items = []
with open('australian_users_items.json', 'r', encoding = 'utf-8') as file:
    for line in file.readlines():
        items.append(ast.literal_eval(line))

df_items = pd.DataFrame(items)

dfitems = df_items


Se normalizan los dataframes

- Se explotan las columnas anidadas para dividirlas en múltiples columnas y se unen al dataframe original


In [ ]:

dfresenias_exploded = dfresenias.explode('reviews')
dfresenias_normalized = pd.json_normalize(dfresenias_exploded['reviews'])
dfresenias_exploded.reset_index(drop=True, inplace=True)
dfresenias_normalized.reset_index(drop=True, inplace=True)

dfresenias_final= pd.concat([dfresenias_exploded.drop('reviews', axis=1), dfresenias_normalized], axis=1)


In [ ]:
dfitems_exploded = dfitems.explode('items')
dfitems_normalized = pd.json_normalize(dfitems_exploded['items'])
dfitems_exploded.reset_index(drop=True, inplace=True)
dfitems_normalized.reset_index(drop=True, inplace=True)

dfitems_final= pd.concat([dfitems.drop('items', axis=1), dfitems_normalized], axis=1)

In [ ]:
juegos_exploded = juegos.explode('items')
juegos_normalized = pd.json_normalize(juegos_exploded['items'])
juegos_exploded.reset_index(drop=True, inplace=True)
juegos_normalized.reset_index(drop=True, inplace=True)

df_juegos_final= pd.concat([juegos.drop('items', axis=1), juegos_normalized], axis=1)

In [ ]:
df_juegos_final.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items_count', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')

In [ ]:
df_juegos_final.shape

(5202150, 22)

Se guardan los datasets provisionales y se escribe el código para leerlo

In [ ]:
#df_juegos_final.to_csv('df_juegos_final.csv')
#dfitems_final.to_csv('dfitems_final.csv')
#dfresenias_final.to_csv('dfresenias_final.csv')


In [ ]:
#df_juegos_final = pd.read_csv('df_juegos_final.csv')
#dfitems_final = pd.read_csv('dfitems_final.csv')
#dfresenias_final = pd.read_csv('dfresenias_final.csv')

Se trabaja un dataset especial para las funciones 1 y 2
- Se limpian las columnas que se utilizarán para no lidiar con NaN, se transforma el tipo de dato de fechas para que sea manejable

In [ ]:
funcion1 = df_juegos_final.dropna(subset=['genres', 'release_date', 'id'])

In [ ]:
#df_juegos_final.drop(columns=['publisher'], inplace=True)
#df_juegos_final.drop(columns=['app_name'], inplace=True)
#df_juegos_final.drop(columns=['discount_price'], inplace=True)
#df_juegos_final.drop(columns=['tags'], inplace=True)
#df_juegos_final.drop(columns=['early_access'], inplace=True)
#df_juegos_final.drop(columns=['specs'], inplace=True)
#df_juegos_final.drop(columns=['price'], inplace=True)
#df_juegos_final.drop(columns=['metascore'], inplace=True)
#df_juegos_final.drop(columns=['developer'], inplace=True)
#df_juegos_final.drop(columns=['items_count'], inplace=True)
#df_juegos_final.drop(columns=['reviews_url'], inplace=True)
#df_juegos_final.drop(columns=['title'], inplace=True)
#df_juegos_final.drop(columns=['url'], inplace=True)
#df_juegos_final.drop(columns=['item_id'], inplace=True)
#df_juegos_final.drop(columns=['playtime_2weeks'], inplace=True)
#df_juegos_final.drop(columns=['steam_id'], inplace=True)
#df_juegos_final.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
funcion1['release_date'] = pd.to_datetime(funcion1['release_date'], errors='coerce')

In [ ]:
release_date_dtype = funcion1['release_date'].dtype

# Print the datatype
print(release_date_dtype)

In [ ]:
funcion1.head(1)

In [ ]:
df_f1_2 = funcion1

Función 1 
-  Indicación: def PlayTimeGenre(genero:str): Debe devolver año con mas horas jugadas para dicho género. Ejemplo de retorno: {"Año con más horas jugadas para Género X" : 2013}

In [ ]:
def PlayTimeGenre(genero):
    genero = genero.lower()
    genero = genero.capitalize()
    df_f1_2['release_date'] = pd.to_datetime(df_f1_2['release_date'], errors='coerce')
    max_horas_ano = None
    max_horas = 0
    horas_por_ano = {}
    
    for index, row in df_f1_2.iterrows():
        if genero in row['genres']:
            # Obtener el año de la fecha de lanzamiento
            year = row['release_date'].year
            
            # Sumar las horas jugadas
            horas_jugadas = row['playtime_forever']
            
            if year not in horas_por_ano:
                horas_por_ano[year] = 0
                
            horas_por_ano[year] += horas_jugadas
            
            if horas_por_ano[year] > max_horas:
                max_horas = horas_por_ano[year]
                max_horas_ano = year
    res = {
    "Año con más horas": max_horas_ano,"Total de horas sumadas": max_horas
    }
            
    return res


In [ ]:
PlayTimeGenre('Action')

{'Año con más horas': 2016, 'Total de horas sumadas': 1825857.0}

In [ ]:
def GetUniqueGenres(funcion1):
    """
    Devuelve una lista con los géneros únicos de la columna `genres`.

    Args:
        funcion_1: El DataFrame que contiene la columna `genres`.

    Returns:
        Una lista con los géneros únicos.
    """

    # Inicializar la variable unique_genres
    unique_genres = []

    # Iterar sobre las filas del DataFrame
    for row in funcion1.itertuples():
        # Obtener la lista de géneros de cada fila
        genres = row.genres

        # Iterar sobre los géneros de cada fila
        for genero in genres:
            # Agregar el género a la lista de géneros únicos
            if genero not in unique_genres:
                unique_genres.append(genero)

    return unique_genres


unique_genres = GetUniqueGenres(funcion_1)

print(unique_genres)


['Action', 'Casual', 'Indie', 'Simulation', 'Strategy', 'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing', 'Massively Multiplayer', 'Early Access', 'Animation &amp; Modeling', 'Video Production', 'Web Publishing', 'Education', 'Software Training', 'Utilities', 'Design &amp; Illustration', 'Audio Production', 'Photo Editing', 'Accounting']


Segunda función
- Indicaciones:  def UserForGenre(genero:str): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
def UserForGenre(genero):
    genero = genero.lower()
    genero = genero.capitalize()
    df_f1_2['release_date'] = pd.to_datetime(df_f1_2['release_date'], errors='coerce')
    max_horas_ano = None
    max_horas = 0
    max_user = None  # Nuevo: mantener un registro del usuario con más horas jugadas
    horas_por_ano = {}
    
    for index, row in df_f1_2.iterrows():
        if genero in row['genres']:
            # Obtener el año de la fecha de lanzamiento
            year = row['release_date'].year
            
            # Sumar las horas jugadas
            horas_jugadas = row['playtime_forever']
            
            if year not in horas_por_ano:
                horas_por_ano[year] = 0
                
            horas_por_ano[year] += horas_jugadas
            
            if horas_por_ano[year] > max_horas:
                max_horas = horas_por_ano[year]
                max_horas_ano = year
                max_user = row['id']
    
    res = {
        "Año con más horas": max_horas_ano,
        "Total de horas sumadas": max_horas,
        "Usuario con más horas jugadas para Género X": max_user 
    }
            
    return res


In [ ]:
UserForGenre('Casual')

{'Año con más horas': 2014,
 'Total de horas sumadas': 1022261.0,
 'Usuario con más horas jugadas para Género X': 261410}

Previo a las funciones 3 y 4, comenzamos con la limpieza del dataframe de reseñas

In [ ]:
dfresenias_final = dfresenias

In [ ]:
dfresenias_final.head(1)

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...


In [ ]:
dfresenias_final.shape

(59305, 5)

In [ ]:
null_id = dfresenias_final['user_id'].isna().sum()
null_posted = dfresenias_final['posted'].isna().sum()
null_item_id = dfresenias_final['item_id'].isna().sum()
null_recommend = dfresenias_final['recommend'].isna().sum()
null_recommend = dfresenias_final['review'].isna().sum()

print(f'Number of null values in user_id: {null_id}')
print(f'Number of null values in posted: {null_posted}')
print(f'Number of null values in item_id: {null_item_id}')
print(f'Number of null values in recommend: {null_recommend}')
print(f'Number of null values in review: {null_recommend}')

Number of null values in user_id: 0
Number of null values in posted: 0
Number of null values in item_id: 0
Number of null values in recommend: 0
Number of null values in review: 0


In [ ]:
dfresenias_final.drop(columns=['last_edited'], inplace=True)
dfresenias_final.drop(columns=['user_url'], inplace=True)
dfresenias_final.drop(columns=['Unnamed: 0'], inplace=True)
dfresenias_final.drop(columns=['helpful'], inplace=True)
dfresenias_final.drop(columns=['funny'], inplace=True)

In [ ]:
#Estas son las columnas que se utilizarán
dfresenias_final = dfresenias_final.dropna(subset=['posted', 'item_id', 'recommend', 'review'])


In [ ]:
#Se analiza el contenido de algunas columnas

unique_values = dfresenias_final['funny'].unique()

with open('unique_values.txt', 'w') as file:
    for value in unique_values:
        file.write(f"{value}\n")

In [ ]:

def keep_last_5_characters(df, column_name):
    dfresenias_final['posted'] = dfresenias_final['posted'].str[-5:]
    return dfresenias


In [ ]:
dfresenias['posted'] = dfresenias['posted'].astype(int)


In [ ]:
dfresenias_final.to_csv('dfresenias_final.csv')

In [ ]:
dfresenias_final = pd.read_csv('dfresenias_final.csv')

Vamos con la limpieza del dataframe de items

In [ ]:
dfitems_final.drop(columns=['playtime_2weeks'], inplace=True)
dfitems_final.drop(columns=['steam_id'], inplace=True)
dfitems_final.drop(columns=['items_count'], inplace=True)
dfitems_final.drop(columns=['user_url'], inplace=True)
dfitems_final.drop(columns=['Unnamed: 0'], inplace=True)
dfitems_final.drop(columns=['playtime_forever'], inplace=True)
dfitems_final.drop(columns=['user_id'], inplace=True)
dfresenias_final.drop(columns=['review'], inplace=True)

In [ ]:
dfitems_final.columns

Index(['user_id', 'item_id', 'item_name'], dtype='object')

In [ ]:
nan_count_per_column = dfitems_final.isna().sum()
nan_count_per_column

item_id      0
item_name    0
dtype: int64

In [ ]:
dfitems_final.shape

(5153209, 2)

In [ ]:
dfitems_final = dfitems_final.dropna(subset=['item_id'])

In [ ]:
dfitems_final['item_id'] = dfitems_final['item_id'].astype(int)

In [ ]:
unique_item_count = dfitems_final['item_name'].nunique()
unique_item_count

10947

In [ ]:
dfitems_final.drop_duplicates(subset=['item_id'], inplace=True)


Combinamos los datasets de items y de reseñas, pues le de items contiene los nombres de los juegos. Se usará el mismo dataset para ambas funciones, 3 y 4

In [ ]:
merged_df = pd.merge(dfresenias_final, dfitems_final, on='item_id', how='left')

In [ ]:
df_f3_4 = merged_df 

Tercera función
- def UsersRecommend(año:int): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales). Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [ ]:
def UsersRecommend(year: int):
    # Verificar si el año es igual a -1 y mostrar un mensaje personalizado
    if year == -1:
        return "El año ingresado es -1, lo cual no es válido."

    # Verificar que el año sea un número entero
    if not isinstance(year, int):
        return "El año debe ser un número entero."

    # Verificar que el año ingresado esté en la columna 'year_integer'
    if year not in df_f3_4['year_integer'].unique():
        return "El año no se encuentra en la columna 'year_integer'."

    # Filtrar el dataset para obtener solo las filas correspondientes al año dado
    juegos_del_año = df_f3_4[df_f3_4['year_integer'] == year]

    # Calcular la cantidad de recomendaciones para cada juego
    recomendaciones_por_juego = juegos_del_año.groupby('item_name')['recommend'].sum().reset_index()

    # Ordenar los juegos por la cantidad de recomendaciones en orden descendente
    juegos_ordenados = recomendaciones_por_juego.sort_values(by='recommend', ascending=False)

    # Tomar los tres primeros lugares
    primer_puesto = juegos_ordenados.iloc[0]['item_name']
    segundo_puesto = juegos_ordenados.iloc[1]['item_name']
    tercer_puesto = juegos_ordenados.iloc[2]['item_name']

    # Crear el diccionario con los tres primeros lugares
    top_tres = {
        "Puesto 1": primer_puesto,
        "Puesto 2": segundo_puesto,
        "Puesto 3": tercer_puesto
    }

    return top_tres

In [ ]:
UsersRecommend(2012)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': "Garry's Mod",
 'Puesto 3': 'Unturned'}

Cuarta función
- def UsersNotRecommend(año:int): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos). Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [ ]:
def UsersNotRecommend(año: int):
    # Verificar si el año es igual a -1 y mostrar un mensaje personalizado
    if año == -1:
        return "El año ingresado es -1, lo cual no es válido."

    # Verificar que el año sea un número entero
    if not isinstance(año, int):
        return "El año debe ser un número entero."

    # Verificar que el año ingresado esté en la columna 'year_integer'
    if año not in df_f3_4['year_integer'].unique():
        return "El año no se encuentra en la columna 'year_integer'."

    # Filtrar el dataset para obtener solo las filas correspondientes al año dado
    juegos_del_año = df_f3_4[df_f3_4['year_integer'] == año]

    # Calcular la cantidad de recomendaciones para cada juego
    recomendaciones_por_juego = juegos_del_año.groupby('item_name')['recommend'].sum().reset_index()

    # Ordenar los juegos por la cantidad de recomendaciones en orden descendente
    juegos_ordenados = recomendaciones_por_juego.sort_values(by='recommend', ascending=True)

    # Tomar los tres primeros lugares
    ultimo_puesto = juegos_ordenados.iloc[0]['item_name']
    penultimo_puesto = juegos_ordenados.iloc[1]['item_name']
    antepenultimo_puesto = juegos_ordenados.iloc[2]['item_name']

    # Crear el diccionario con los tres primeros lugares
    ultimos_tres = {
        "Puesto 1": ultimo_puesto,
        "Puesto 2": penultimo_puesto,
        "Puesto 3": antepenultimo_puesto
    }

    return ultimos_tres

In [ ]:
UsersNotRecommend(2012)

{'Puesto 1': 'Pixel Puzzles: Japan',
 'Puesto 2': 'James Cameron’s Avatar™: The Game',
 'Puesto 3': 'Zero Punctuation: Hatfall - Hatters Gonna Hat Edition'}

Previo a la quinta función, se realiza en Análisis de sentimiento

In [ ]:
# Crear una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Supongamos que df_reviews2 es tu DataFrame y 'reviews' es la columna que quieres analizar
for index, row in dfresenias_final.iterrows():
    text = str(row['review'])
    sentiment_scores = sia.polarity_scores(text)

    # Extraer los puntajes de sentimiento
    compound_score = sentiment_scores['compound']

    # Puedes hacer algo con el puntaje compuesto aquí
    if compound_score >= 0.05:
        sentiment_label = 'Positivo'
    elif compound_score <= -0.05:
        sentiment_label = 'Negativo'
    else:
        sentiment_label = 'Neutral'

    # Guardar el resultado en una nueva columna o hacer lo que necesites
    dfresenias_final.at[index, 'sentiment_analysis'] = sentiment_label

# Ahora, tu DataFrame df_reviews tiene una nueva columna llamada 'sentiment' con las etiquetas de sentimiento

value_counts = dfresenias_final['sentiment_analysis'].value_counts()

# Create a bar plot
plt.bar(value_counts.index, value_counts.values)

# Set labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Bar Plot of Values')

# Show the plot
plt.show()

In [ ]:
# Fusionar los DataFrames con sufijos
df_final = pd.merge(df_juegos_final, merged_df, on='item_id', how='inner', suffixes=('_left', '_right'))

In [ ]:
df_final.to_csv("df_final.csv")

In [ ]:
df_final.shape

(2725615, 13)

In [ ]:
df_final.dropna(inplace=True)

In [ ]:
df_final['year'] = df_final['release_date'].dt.year


In [ ]:
df_f5 = df_final

Quinta función
- def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento. Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [ ]:
def sentiment_analysis(año : int):
    # Filtrar el DataFrame por el año proporcionado
    df_filtered = df_f5[df_f5['year'] == año]
    
    # Contar la cantidad de registros por cada análisis de sentimiento
    resultados = df_filtered['sentiment_analysis'].value_counts()
    
    # Si algún análisis de sentimiento está ausente, añádelo con 0
    for sentimiento in ['Negativo', 'Neutral', 'Positivo']:
        if sentimiento not in resultados:
            resultados[sentimiento] = 0
    
    # Almacenar los resultados en una lista de tuplas
    resultados_lista = [(sentimiento, cantidad) for sentimiento, cantidad in resultados.items()]
    
    return resultados_lista

In [ ]:
sentiment_analysis(2011)

EDA


- Se buscan outliers en variables de cantidad, solo se encuentra en 'playtime_forever' usando .describe() y se eliminan los mayores a 6500 horas de juego ya que es muy poco probable que alguien dedique tantas horas de juego diarias a lo largo de 5 años

In [ ]:
filtered_values = df_1[df_1['playtime_forever'] > 5000]

filtered_values


,Unnamed: 0,genres,title,release_date,reviews_url,id,user_id,steam_id,item_id,item_name,playtime_forever
15,15,"['Free to Play', 'Indie', 'Simulation', 'Sports']",SNOW - Lifetime Pack,2018-01-03,http://steamcommunity.com/app/774279/reviews/?...,774279,NaN,NaN,219830.0,King Arthur's Gold,33460.0
17,17,"['Racing', 'Simulation', 'Sports']",RC Plane 3 - Canyon Scenario,2017-12-22,http://steamcommunity.com/app/775880/reviews/?...,775880,NaN,NaN,211820.0,Starbound,31000.0
24,24,['Action'],Twisted Enhanced Edition,2018-01-01,http://steamcommunity.com/app/690410/reviews/?...,690410,NaN,NaN,298630.0,The Escapists,5063.0
26,26,"['Action', 'Casual', 'Indie', 'Massively Multi...",Tactics Forever,2018-01-01,http://steamcommunity.com/app/413120/reviews/?...,413120,NaN,NaN,302590.0,Of Guards And Thieves,9040.0
40,40,['Action'],Advent Rising,2005-08-09,http://steamcommunity.com/app/3800/reviews/?br...,3800,NaN,NaN,214850.0,GameMaker: Studio,10617.0
...,...,...,...,...,...,...,...,...,...,...,...
28511,28511,"['Action', 'Adventure', 'Casual', 'Indie', 'Ma...",PLAYERUNKN4WN: Zombie,2018-01-01,http://steamcommunity.com/app/737750/reviews/?...,737750,NaN,NaN,730.0,Counter-Strike: Global Offensive,13557.0
28521,28521,['Indie'],NeoBalls,2018-01-03,http://steamcommunity.com/app/771610/reviews/?...,771610,NaN,NaN,4000.0,Garry's Mod,7933.0
28527,28527,['Action'],Deathmatch Classic,2001-06-01,http://steamcommunity.com/app/40/reviews/?brow...,40,NaN,NaN,105600.0,Terraria,20961.0
28532,28532,"['Action', 'Casual', 'Indie']",Chicken Shoot Gold,2003-11-01,http://steamcommunity.com/app/259340/reviews/?...,259340,NaN,NaN,730.0,Counter-Strike: Global Offensive,14257.0


In [ ]:
df_1.describe(include='all') 

In [ ]:
df_f3.describe(include='all') 

In [ ]:
df_f5.describe(include='all') 

In [ ]:
df_1_2 = df_f1_2[df_f1_2['playtime_forever'] <= 6500]

Los 3 datasets están limpios

In [ ]:
df_1_2
df_f3_4
df_f5

Modelo de ML

- Se prueba con una nube de palabras, pero la cercanía entre los juegos los pone o en la misma saga o en juegos que si bien tienen algo en común en la temárica, realmente no comparten mucho en cuanto al contenido, estructura y otros rasgos del juego

In [ ]:
def get_matching_words(input_letters, column_name):
    matching_words = []

    for word in df_f1_2[column_name]:
        if word[:4] == input_letters:
            matching_words.append(word)

    return matching_words

# Assuming 'df' is your DataFrame and 'word_column' is the name of the column with words
# Replace 'df' and 'word_column' with your actual DataFrame and column name.


get_matching_words('Kill', 'title')



- Se limpia un nuevo dataset (el relacionado a juegos), creado especialmente para el modelo de Machine Learning, debido a que necesita ser uno compacto y con rasgos muy particulares para eficientar su uso

In [ ]:
df_juegos_ml = df_juegos_final

In [ ]:
df_juegos_ml.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items_count'],
      dtype='object')

In [ ]:
# Obtén el recuento de NaN por columna
nan_count = df_juegos_ml.isna().sum()

# Muestra el recuento de NaN por columna
print(nan_count)

In [ ]:
df_juegos_ml = df_juegos_ml.drop(['app_name', 'publisher', 'url', 'release_date', 'reviews_url', 'discount_price', 'price', 'early_access', 'id', 'metascore', 'developer', 'steam_id', 'items_count'], axis=1)

In [ ]:
df_juegos_ml = df_juegos_ml.drop(['tags', 'user_id'], axis=1)

In [ ]:
df_juegos_ml = df_juegos_ml.drop(['Unnamed: 0.2', 'Unnamed: 0.2', 'Unnamed: 0'], axis=1)

In [ ]:
df_juegos_ml.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'genres', 'title'], dtype='object')

In [ ]:
df_juegos_ml.dropna(subset=['genres', 'title', 'tags', 'specs', 'user_id'], inplace=True)


In [ ]:
df_juegos_ml.dropna(subset=['genres', 'title', 'tags', 'specs'], inplace=True)


In [ ]:
unique_values = df_juegos_ml['title'].unique()

# Print the unique values
print(len(unique_values))

28520


In [ ]:
df_juegos_ml.shape

(28543, 6)

In [ ]:
df_juegos_ml.to_csv("df_juegos_ml.csv")

In [ ]:
df_juegos_ml = pd.read_csv("df_juegos_ml.csv")

- Se configura el modelo de Machine Learning basándonos en la similaridad del coseno

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import IncrementalPCA

In [ ]:
# Paso 1: Preprocesamiento y vectorización
textos = df_juegos_ml['genres']
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(textos)

# Paso 2: Reducción de dimensionalidad con PCA
ipca = IncrementalPCA(n_components=25)  # Por ejemplo, reduce a 50 componentes
matriz_ipca = ipca.fit_transform(matriz_tfidf)


# Paso 3: Cálculo de similitud del coseno después de PCA
similitud_pca = cosine_similarity(matriz_ipca)


In [ ]:
similitud = similitud_pca

In [ ]:
def obtener_recomendaciones(titulo, n=5):
    indice_titulo = df_juegos_ml[df_juegos_ml['title'] == titulo].index[0]
    similitudes_titulo = similitud[indice_titulo]
    indices_recomendaciones = similitudes_titulo.argsort()[-n-1:-1][::-1]
    recomendaciones = df_juegos_ml.loc[indices_recomendaciones, 'title']
    
    # Convierte la serie en un diccionario
    recomendaciones_dict = recomendaciones.to_dict()

    return recomendaciones_dict

In [ ]:
filas_seleccionadas = df_juegos_final[df_juegos_final['title'] == 'The Elder Scrolls®: Legends™']
filas_seleccionadas


,Unnamed: 0,genres,title,specs
15320,105774,['Free to Play'],The Elder Scrolls®: Legends™,"['Single-player', 'Multi-player', 'Online Mult..."


Resultados de ejemplo 1:

In [ ]:
'''
0	    88310	['Action', 'Casual', 'Indie', 'Simulation', 'S...'	      Lost Summoner Kitty	                        ['Single-player']

11489	101249	['Action', 'Casual', 'Indie', 'Simulation', 'S...'	       Wooden Battles      	                        ['Single-player']
10697	100329	['Action', 'Casual', 'Simulation', 'Strategy']	   Trivia Vault: Video Game Trivia Deluxe	            ['Single-player']
2351	90729	['Casual', 'Indie', 'Simulation', 'Strategy']	      Wildlife Park 2 - Farm World 	                    ['Single-player']
8991	98347	['Casual', 'Indie', 'Simulation', 'Strategy']	    Weather Lord: In Search of the Shaman	            ['Single-player']
2438	90819	['Casual', 'Indie', 'Simulation', 'Strategy']	            GamersGoMakers	                            ['Single-player']
'''

Resultados del ejemplo 2:

In [ ]:
'''
1	    88311	['Free to Play', 'Indie', 'RPG', 'Strategy']	              Ironbound	                            ['Single-player', 'Multi-player', 'Online Mult...

17863	108873	['Adventure', 'Free to Play', 'RPG', 'Strategy']	          Spellstone	                        ['Single-player', 'Online Multi-Player', 'Cros...
15587	106116	['Adventure', 'Free to Play', 'RPG', 'Strategy']	Animation Throwdown: The Quest for Cards	    ['Single-player', 'Online Multi-Player', 'Cros...
2351	90729	['Casual', 'Indie', 'Simulation', 'Strategy']	        Wildlife Park 2 - Farm World 	            ['Single-player']
12405	102331	['Free to Play', 'Strategy']	                            Yu-Gi-Oh! Duel Links                    ['Single-player', 'Online Multi-Player', 'Cros...
15320	105774	['Free to Play']	                                   The Elder Scrolls®: Legends™                 ['Single-player', 'Multi-player', 'Online Mult...
'''

Sin embargo, este modelo necesita mucha memoria RAM, por lo que se opta por uno menos pesado porque se va a deployar en Render
- Primero se limpia el dataframe para luego ir al modelo y finalmente a la función

In [ ]:
df_juegos_ml['title'] = df_juegos_ml['title'].str.capitalize()

df_juegos_ml.head(1)

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,genres,title,genres_encoded
0,0,0,0,"['action', 'casual', 'indie', 'simulation', 's...",Lost summoner kitty,225


In [ ]:
count_values = df_juegos_ml.nunique()

print(count_values)

genres              847
title             28510
genres_encoded      847
dtype: int64


In [ ]:
for col in df_juegos_ml.columns:
    has_duplicates = df_juegos_ml[col].duplicated().any()
    if has_duplicates:
        print(f'La columna {col} tiene valores duplicados.')
    else:
        print(f'La columna {col} no tiene valores duplicados.')

La columna genres tiene valores duplicados.
La columna title no tiene valores duplicados.
La columna genres_encoded tiene valores duplicados.


In [ ]:
nan_counts = df_juegos_ml.isna().sum()

print(nan_counts)

genres            0
title             0
genres_encoded    0
dtype: int64


In [ ]:
df_juegos_ml = df_juegos_ml.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
df_juegos_ml['title'] = df_juegos_ml['title'].str.strip()
df_juegos_ml = df_juegos_ml.drop_duplicates(subset=['title'])


In [ ]:
df_juegos_ml['title'] = df_juegos_ml['title'].astype(str)
df_juegos_ml = df_juegos_ml.drop_duplicates(subset=['title'])


In [ ]:
df_juegos_ml['title'].fillna('', inplace=True)
df_juegos_ml = df_juegos_ml.drop_duplicates(subset=['title'])


In [ ]:
df_juegos_ml['title'] = df_juegos_ml['title'].str.lower()
df_juegos_ml = df_juegos_ml.drop_duplicates(subset=['title'])


In [ ]:
df_juegos_ml = df_juegos_ml.drop_duplicates(subset=['title'])


In [ ]:
df_juegos_ml.to_csv("df_juegos_ml.csv")

In [3]:
df_final = pd.read_csv('df_juegos_ml.csv')

In [2]:
df_final = pd.read_csv("df_final.csv")

In [5]:
df_ml = df_final.drop(['Unnamed: 0', 'Unnamed: 0_left','release_date', 'reviews_url','id', 'item_name_left','playtime_forever', 'user_id','posted', 'recommend', 'sentiment_analysis'], axis=1)

In [12]:
df_ml = df_ml.drop_duplicates(subset='item_id')

# Print the results
df_ml.shape

(2318, 3)

In [13]:
df_ml.to_csv("df_ml.csv")

Modelo de Machine Learning

In [21]:

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

# Crear una instancia del codificador
label_encoder = LabelEncoder()

# Cargar los datos
df_ml = pd.read_csv("df_ml.csv")
df_ml["genres_encoded"] = label_encoder.fit_transform(df_ml["genres"])

# Crear un diccionario de los títulos asociados a cada item_id
titles_by_item_id = {}
for i in range(len(df_ml)):
    titles_by_item_id[df_ml.loc[i, "item_id"]] = df_ml.loc[i, "title"]

# Crear el modelo K-Nearest Neighbors
k = 5
model = KNeighborsClassifier(n_neighbors=k)

# Entrenar el modelo
model.fit(df_ml[['genres_encoded']], df_ml['title'])



KNeighborsClassifier()

In [22]:
# Crear una función para obtener las recomendaciones
def get_recommendations(item_id: int):
    # Buscar el género codificado del juego proporcionado por el usuario
    input_game = df_ml[df_ml["item_id"] == item_id]["genres_encoded"].values[0]

    # Encontrar los juegos más similares
    _, indices = model.kneighbors([[input_game]])

    # Obtener los títulos de los juegos similares
    similar_games = [titles_by_item_id[df_ml.loc[i, "item_id"]] for i in indices[0]]

    # Devolver un diccionario de los títulos
    return {"similar_games": similar_games}

In [23]:
get_recommendations(228440)

c:\Users\German\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


{'similar_games': ['Lost Summoner Kitty',
  'Crankies Workshop: Grizzbot Assembly',
  'Trivia Vault: Mixed Trivia',
  'World of Cinema - Directors Cut',
  "Trivia Vault: 1980's Trivia 2"]}